In [ ]:
test = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""


In [ ]:
# I hope you're happy Harvey
with open("inputs/Day10_input.txt", "r") as file:
    actual = file.read()

In [ ]:
def set_l(dataset):
    l = [x.split(" ") for x in dataset.split("\n")]

    for i in range(len(l)):
        l[i][0] = [x == "#" for x in l[i][0][1:-1]]
        for j in range(1, len(l[i]) - 1):
            l[i][j] = [int(x.strip()) for x in l[i][j][1:-1].split(",")]

        l[i][-1] = [int(x) for x in l[i][-1][1:-1].split(",")]

    return l


# PART 1

In [ ]:
l = set_l(test)

l

In [ ]:
def press_button(states, button):
    for n in button:
        states[n] = not states[n]

    return states

In [ ]:
def press_buttons(states, buttons_to_press, buttons):
    for press, button in zip(buttons_to_press, buttons):
        if press:
            states = press_button(states, button)

    return states

In [ ]:
import itertools

tot_presses = 0

for row in l:
    target = row[0]
    buttons = row[1:-1]
    fewest_buttons = len(buttons)

    options = list(itertools.product([0, 1], repeat=len(buttons)))

    for option in options:
        # print(option)
        if sum(option) < fewest_buttons:
            states = press_buttons([0] * len(target), option, buttons)
            # print(f"{states = }. {target = }")

            if states == target:
                fewest_buttons = sum(option)
                # print(fewest_buttons)

    tot_presses += fewest_buttons

tot_presses

# PART 2 - simultaneous equations

shoutout to the reddit community for pointing me towards the solution, and to AI for writing part of it

In [110]:
import numpy as np
from scipy.optimize import Bounds, LinearConstraint, milp


# thanks to AI for writing my simultaneous equation solver
def calculate_min_presses(buttons, target):
    max_num = max([max(x) for x in buttons])

    width = max_num + 1

    coeff_matrix = np.array(
        [[1 if col in row else 0 for col in range(width)] for row in buttons]
    )
    # Transpose so columns are variables (buttons), rows are equations (targets)
    A = coeff_matrix.T  # shape (6, 4)
    num_buttons = A.shape[1]

    # Objective: Minimize sum(x)
    c = np.ones(num_buttons)

    # Constraints: A @ x = target
    linear_constraint = LinearConstraint(A, target, target)  # equality

    # Bounds: x >= 1 (positive integer only)
    bounds = Bounds([0] * num_buttons, [np.inf] * num_buttons)

    # Variable integrality (all variables must be integer)
    integrality = np.ones(num_buttons, dtype=int)

    # Solve
    result = milp(
        c=c, constraints=[linear_constraint], bounds=bounds, integrality=integrality
    )

    if result.success:
        solution = np.round(result.x).astype(int)
        # print("Button presses (minimal positive integer):", solution)
        # print("Total presses:", solution.sum())
        return solution.sum()
    else:
        print("No solution found! Status:", result.status)

In [114]:
l = set_l(actual)

total_button_presses = 0

for row in l:
    target = np.array(row[-1])
    buttons = row[1:-1]

    total_button_presses += calculate_min_presses(buttons, target)
    # print(total_button_presses)


print(f"{total_button_presses = }")

total_button_presses = np.int64(18387)


# PART 2 - "CORRECT" CODE BUT IT WILL NEVER RUN! 

In [ ]:
def sum_combinations(L, N):
    """
    Returns all lists of length L of non-negative integers that sum to N.
    Written by AI
    """
    if L == 1:
        return [[N]]
    result = []
    for i in range(N + 1):
        for tail in sum_combinations(L - 1, N - i):
            result.append([i] + tail)
    return result


# Example:
print(sum_combinations(3, 3))

In [ ]:
def press_button_2(states, button):
    for n in button:
        states[n] += 1

    return states

In [ ]:
def press_buttons_2(states, buttons_to_press, buttons):
    for press, button in zip(buttons_to_press, buttons):
        for p in range(press):
            states = press_button_2(states, button)

    return states

In [ ]:
def calc_max(target, buttons):
    maxes = [max(target)] * len(buttons)
    for it, t in enumerate(target):
        for ib, b in enumerate(buttons):
            if it in b:
                if t < maxes[ib]:
                    maxes[ib] = t

    print(f"{target = }. {buttons = }. {maxes = }.")

    return maxes


calc_max(l[0][-1], l[0][1:-1])

In [ ]:
l = set_l(actual)
# l

In [ ]:
import itertools

tot_presses = 0

for iterx, row in enumerate(l):
    target = row[-1]
    buttons = row[1:-1]
    reached_target = False
    button_presses = max(target) - 1
    print(target)

    max_values = calc_max(target, buttons)

    while not reached_target:
        button_presses += 1
        print(button_presses)

        all_options = sum_combinations(len(buttons), button_presses)
        filtered_options = []
        print(f"{len(all_options) = }")

        for o in all_options:
            if all([a <= b for a, b in zip(o, max_values)]):
                filtered_options.append(o)

        print(f"{len(all_options) = }. {len(filtered_options) = }")
        for option in filtered_options:
            states = [0] * len(target)

            for btn_idx, btn_count in enumerate(option):
                for b in buttons[btn_idx]:
                    states[b] += btn_count

            if states == target:
                fewest_buttons = sum(option)
                reached_target = True
                print(fewest_buttons)

    tot_presses += button_presses
    print(f"Completed {iterx + 1}/{len(l)} ({((iterx + 1) / len(l) * 100):.2f}%)")

tot_presses